# PyVista - NASA P3 Case Directory Read & Analysis

In [1]:
import numpy as np
import matplotlib.pyplot as plt

import pyvista as pv

## Define Helper Functions to Slice the Internal Mesh

In [48]:
def slice_center(mesh, NORMAL='z', ORIGIN=[0, 0, 0], translate=False, CONTOUR=False):
    """Slice mesh through center in normal direction, move to zero normal."""
    slice_mesh = mesh.slice(normal=NORMAL, origin=ORIGIN, contour=CONTOUR)
    if translate is not False:
        if NORMAL == 'z':
            slice_mesh.translate((0, 0, -slice_mesh.center[-1]), inplace=True)
        elif NORMAL == 'x':
            slice_mesh.translate((-slice.mesh.center[0], 0, 0), inplace=True)
        elif NORMAL == 'y':
            slice_mesh.translate((0, -slice.mesh.center[1], 0), inplace=True)
        else:
            print('ERROR: Normal not found - ', NORMAL)
            slice_mesh = None
    return slice_mesh

In [57]:
def slice_multiple(mesh, NORMAL='z', nslices=2, CONTOUR=False):
    """Slice mesh multiple times along specified axis, move to z=0."""
    slice_mesh = mesh.slice_along_axis(n=nslices, axis=NORMAL, contour=CONTOUR)
    #slice_mesh.translate((0, 0, -slice_mesh.center[-1]), inplace=True)
    return slice_mesh

## Define the Case Directory

In [3]:
case_directory = '/Users/jrobrien/Dissertation/data/solutions/NASA_noPylons_v2_tas120_aoa0_900T33/'

In [4]:
reader = pv.POpenFOAMReader(case_directory + 'NASA_noPylons_v2_tas120_aoa0_900T33.OpenFOAM')

## Subset the final run 

In [5]:
reader.time_values[-1]

786.0

In [6]:
%%time
reader.set_active_time_value(reader.time_values[-1])
reader.cell_to_point_creation = True
mesh = reader.read()

CPU times: user 4.64 s, sys: 705 ms, total: 5.35 s
Wall time: 5.33 s


In [7]:
internal_mesh = mesh["internalMesh"]
boundaries = mesh["boundary"]

In [8]:
internal_mesh

UnstructuredGrid (0x105f12e20)
  N Cells:    10691736
  N Points:   11526824
  X Bounds:   -2.000e+02, 2.000e+02
  Y Bounds:   -2.000e+02, 2.000e+02
  Z Bounds:   -2.000e+02, 2.000e+02
  N Arrays:   18

## Slice Mesh to View Aircraft - Plot

In [9]:
slice_internal_mesh = slice_z_center(internal_mesh)

2023-05-22 21:27:20.283 (  22.680s) [          2C0622]      vtkPolyhedron.cxx:1742  WARN| A cell with a non-manifold triangulation has been encountered. This cell cannot be contoured.
2023-05-22 21:27:20.368 (  22.766s) [          2C0622]      vtkPolyhedron.cxx:1742  WARN| A cell with a non-manifold triangulation has been encountered. This cell cannot be contoured.


In [10]:
slice_internal_mesh

PolyData (0x105f12ee0)
  N Cells:    56461
  N Points:   82061
  N Strips:   0
  X Bounds:   -2.000e+02, 2.000e+02
  Y Bounds:   -2.000e+02, 2.000e+02
  Z Bounds:   -6.805e-07, 6.805e-07
  N Arrays:   18

In [12]:
pt = pv.Plotter()
pt.add_mesh(slice_internal_mesh, scalars='U')
pt.show(cpos="xy")

Widget(value="<iframe src='http://localhost:65168/index.html?ui=P_0x3be38fdf0_1&reconnect=auto' style='width: …

## Additional Slicing Functions

### Z-slice - move origin

In [52]:
slice_mesh2 = slice_center(mesh['internalMesh'], NORMAL='z', CONTOUR=True)
pt = pv.Plotter()
pt.add_mesh(slice_mesh2, scalars='U')
pt.show(cpos="xy")

2023-05-22 22:25:05.321 (3487.792s) [          2C0622]      vtkPolyhedron.cxx:1742  WARN| A cell with a non-manifold triangulation has been encountered. This cell cannot be contoured.
2023-05-22 22:25:05.407 (3487.878s) [          2C0622]      vtkPolyhedron.cxx:1742  WARN| A cell with a non-manifold triangulation has been encountered. This cell cannot be contoured.


Widget(value="<iframe src='http://localhost:65168/index.html?ui=P_0x3c6562460_18&reconnect=auto' style='width:…

### y-axis slice

In [35]:
slice3 = slice_center(mesh['internalMesh'], NORMAL='y')

In [42]:
slice3 = slice_center(mesh['internalMesh'], NORMAL='y', ORIGIN=[0, 0, 5])
pt = pv.Plotter()
pt.add_mesh(slice3, scalars='U')
pt.show(cpos="zx")

Widget(value="<iframe src='http://localhost:65168/index.html?ui=P_0x3c655e220_10&reconnect=auto' style='width:…

### x-slice

In [50]:
slice4 = slice_center(mesh['internalMesh'], NORMAL='x', ORIGIN=[15, 0, 0], CONTOUR=True)
pt = pv.Plotter()
pt.add_mesh(slice4, scalars='U')
pt.show(cpos="zy")

Widget(value="<iframe src='http://localhost:65168/index.html?ui=P_0x3c6562d30_17&reconnect=auto' style='width:…

### Polyslice

In [62]:
sliceA = slice_multiple(mesh['internalMesh'], NORMAL='z', nslices=10, CONTOUR=True)
sliceB = slice_multiple(mesh['internalMesh'], NORMAL='x', nslices=10, CONTOUR=True)
sliceB = slice_multiple(mesh['internalMesh'], NORMAL='y', nslices=10, CONTOUR=True)
pt = pv.Plotter()
pt.add_mesh(sliceA + sliceB + sliceC, scalars='U')
pt.show(cpos="xy")

TypeError: unsupported operand type(s) for +: 'MultiBlock' and 'MultiBlock'

## Thresholding - Not working

In [74]:
threshed = mesh['internalMesh'].threshold(value=120)